
&nbsp;

# 14. Réduction via PCA/EOF

---

&nbsp;

## 1. Rappel et objectif

Ce notebook, à la différence des précédents, n'a pas de but analytique. L'objectif est de construire concrètement l’état réduit retenu et le sauvegarder dans un format standardisé pour les étapes ultérieures.

&nbsp;

> #### Quel est l'état à retenir ?

Rappelons en condensé ce qui a été énoncé dans le notebook 10.

En l'état, nous démarrons le pré-traitement avec un champs spatial de SST qui évolue dans le temps avec des milliers de points spatiaux corrélés entre eux. 

La nature géophysique des champs avec lesquels nous travaillons nous avait amené à penser que l'EOF serait la méthode adaptée pour réduire notre série d'anomalies de SST désaisonnalisée. On transforme ainsi notre matrice actuelle de la manière suivante :

$$ X(x, y, t) \approx \overset{K}{\underset{k=1}{\sum}}a_k(t)\phi_k(x,y) $$

La réduction de dimensionnalité est nécessaire pour clarifier ce qu'on veut faire apprendre à notre futur modèle. Je cite la partie 1. du notebook 10 : "Sans cela on s'expose à un apprentissage instable, une sur-paramétrisation massive et une interprétation impossible. Le modèle ne doit apprendre ni le bruit, ni la grille, mais la dynamique.". Suivant l'EOF, on obtient donc un vecteur d'état composait des PCs (principaux composants) de la réduction, soit :

$$
\dot{a}_k(t) = f(\begin{pmatrix} a_1(t) \\ a_2(t) \\ \vdots \\ a_k(t) \end{pmatrix})
$$

Finalement, on fournit l'état dynamique réduit suivant au modèle SciML :

$$
\dot{a}_k(t) = \begin{pmatrix} \dot{a}_1(t) \\ \dot{a}_2(t) \\ \vdots \\ \dot{a}_1(t) \end{pmatrix} = f_\theta(a_1(t), a_2(t),..., a_k(t))
$$

&nbsp;

---

&nbsp;

## 2. Construction de l’état réduit par EOF

Nous avons déjà réalisé les vérifications et discuter de la théorie derrière la méthode de réduction de dimensionnalité PCA/EOF (voir Notebook 10).

Appliquons réellement la méthode au jeu de donnée, avec les paramètres choisies préalablement (~100 modes, SingularValueDecomposition Solver en mode "full", EOF) et sauvegardons le résultat dans notre répertoire *data/processed/*.

Voir le lien suivant pour plus de précisions sur la fonction *PCA* de *sklearn.decomposition* utilisée.

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA

&nbsp;


In [2]:
import numpy as np
import xarray as xr
from sklearn.decomposition import PCA

# Load deseasonalized SST data
ds = xr.open_dataset("data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")

# Extract SST data and reshape for PCA
sst = ds["analysed_sst"]
sstStacked = sst.stack(space=("latitude", "longitude"))
sstStacked = sstStacked.dropna("space")

X = sstStacked.values  # shape (Nt, Nspace)

K = 150  # Number of principal components to retain plus some margin if we want to test more later

pca = PCA(n_components=K, svd_solver="full") # We apply PCA with K components and full SVD solver so we respect timeStep/numberSpatialPoints ratio
PCs = pca.fit_transform(X)        # a_k(t)
EOFs = pca.components_            # phi_k(x)
explainedVar = pca.explained_variance_ratio_

# Next step, we save PCs and EOFs in an appropriate formated xarray Dataset

# We reconstruct EOFs into original spatial shape with NaNs where data was missing
template = sst.isel(time=0)

# Create DataArray for EOFs with NaNs in missing data locations
EOFsDa = xr.DataArray(
    np.full((K , template.sizes["latitude"], template.sizes["longitude"]), np.nan), # K, lat, lon as shape filled with NaNs
    dims=("mode", "latitude", "longitude"), # dimension names in the CDF
    coords={
        "mode": np.arange(K), # filling with mode numbers
        "latitude": template["latitude"], # filling with latitude values
        "longitude": template["longitude"], # filling with longitude values
    },
)

EOFsDa.values.reshape(K, -1)[:, ~np.isnan(sst.isel(time=0).values.flatten())] = EOFs # This line fills the EOFsDa with EOF values at non-NaN locations

# Then we just form a Dataset with PCs, EOFs and explained variance
dsOut = xr.Dataset(
    {
        "PCs": (("time", "mode"), PCs),
        "EOFs": EOFsDa,
        "explainedVar": (("mode",), explainedVar),
    },
    coords={
        "time": sst["time"],
        "mode": np.arange(K),
    },
)

# Finally we save the Dataset to a netCDF file
dsOut.to_netcdf("data/processed/sstReducedStateCOPERNICUS20102019.nc")

print(dsOut) # See magic

<xarray.Dataset> Size: 12MB
Dimensions:       (time: 3652, mode: 150, latitude: 60, longitude: 140)
Coordinates:
  * time          (time) datetime64[ns] 29kB 2010-01-01 ... 2019-12-31
  * mode          (mode) int64 1kB 0 1 2 3 4 5 6 ... 143 144 145 146 147 148 149
  * latitude      (latitude) float32 240B 48.03 48.08 48.13 ... 50.93 50.98
  * longitude     (longitude) float32 560B -4.975 -4.925 -4.875 ... 1.925 1.975
    month         (time) int64 29kB ...
Data variables:
    PCs           (time, mode) float32 2MB -13.05 -1.123 ... 0.5583 -0.3348
    EOFs          (mode, latitude, longitude) float64 10MB 0.01074 ... 0.03121
    explainedVar  (mode) float32 600B 0.8209 0.04989 ... 5.848e-05 5.835e-05


&nbsp;

On obtient finalement la structure de fichier suivante comme entrée du SciML.

&nbsp;

**Structure du fichier *sstReducedStateCOPERNICUS20102019.nc***

- PCs(time, mode) → coefficients temporels 

- EOFs(mode, latitude, longitude) → modes spatiaux associés aux PCs

- explainedVar(mode) → variance expliquée par chaque mode

- coords :

    - time : axe temporel original

    - latitude, longitude : grille spatiale

    - mode : index des composantes EOF

&nbsp;

---

&nbsp;

## 3. Alternatives de réduction

#### Later
